In [1]:
import torch
from det3d.torchie import Config
from det3d.models import build_detector
from det3d.torchie.trainer import load_checkpoint

config = "/workspace/centerformer/configs/nusc/nuscenes_centerformer_poolformer.py"

cfg = Config.fromfile(config)
checkpoint_path = "/workspace/centerformer/work_dirs/nuscenes_poolformer/poolformer.pth"

model = build_detector(cfg.model, train_cfg=None, test_cfg=cfg.test_cfg)
checkpoint = load_checkpoint(model, checkpoint_path, map_location="cpu")

Deformable Convolution not built!
Deformable Convolution not built!
Use HM Bias:  -2.19


/data/centerformer/det3d/core/bbox/geometry.py:149: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit(nopython=False)
/data/centerformer/det3d/core/bbox/geometry.py:162: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit(nopython=False)
/data/centerformer/det3d/core/bbox/geometry.py:280: NumbaDeprecationWarning: The 'nopython' keyword argument was

In [2]:
from torch import nn

class CenterFinder(nn.Module):
    def __init__(self, model):
        super(CenterFinder, self).__init__()
        self.model = model

    def forward(self, x):
        # x = self.model.neck(x)
        # preds = self.model.bbox_head(x)
        # return preds
        return self.model.neck.find_centers(x) # pos_features, out_dict_list, ct_feat 
    
centerFinder = CenterFinder(model)
centerFinder.cuda()
centerFinder.eval();

In [3]:
import pickle
# with open("/workspace/centerformer/work_dirs/partition/sample_data/x.pkl", 'wb') as handle:
#     pickle.dump(x, handle)

pickle_dir = "/workspace/centerformer/work_dirs/partition/sample_data/"

with open(pickle_dir + "example.pkl", 'rb') as handle:
    example = pickle.load(handle)

In [4]:
with torch.no_grad():
    output = centerFinder.model.reader(example['points'])    
    voxels, coors, shape = output
    
with open(pickle_dir + "reader_output.pkl", 'rb') as handle:
    gt = pickle.load(handle)
print(voxels[:5])
print(gt[0][:5])
print(coors[:5])
print(gt[1][:5])

tensor([[ 3.4004, -7.6160, -4.9076,  3.0000,  0.0000],
        [ 3.4309, -7.5684, -4.8892,  5.0000,  0.0000],
        [ 3.4336, -7.6233, -4.7427,  4.0000,  0.0752],
        [ 4.6929, -7.6385, -4.2133,  3.0000,  0.0000],
        [ 4.6846, -7.7457, -4.0402, 23.0000,  0.1253]], device='cuda:0')
tensor([[ 3.4004, -7.6160, -4.9076,  3.0000,  0.0000],
        [ 3.4309, -7.5684, -4.8892,  5.0000,  0.0000],
        [ 3.4336, -7.6233, -4.7427,  4.0000,  0.0752],
        [ 4.6929, -7.6385, -4.2133,  3.0000,  0.0000],
        [ 4.6846, -7.7457, -4.0402, 23.0000,  0.1253]], device='cuda:0')
tensor([[  0,   0, 618, 765],
        [  0,   0, 619, 765],
        [  0,   1, 618, 765],
        [  0,   3, 618, 782],
        [  0,   4, 616, 782]], device='cuda:0')
tensor([[  0,   0, 618, 765],
        [  0,   0, 619, 765],
        [  0,   1, 618, 765],
        [  0,   3, 618, 782],
        [  0,   4, 616, 782]], device='cuda:0')


In [5]:
with torch.no_grad():
    x, _ = centerFinder.model.backbone(voxels,coors, len(example['points']), shape)

In [6]:
with open(pickle_dir + "findcenter_input.pkl", 'rb') as handle:
    x = pickle.load(handle)

with torch.no_grad():
    ct_feat, center_pos_embedding, out_dict_list = centerFinder(x)
    
with open(pickle_dir + "findcenter_output1.pkl", 'rb') as handle:
    gt = pickle.load(handle)

print((ct_feat==gt).all())

tensor(True, device='cuda:0')


/root/anaconda3/envs/centerformer/lib/python3.9/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ../aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


In [7]:
with torch.no_grad():
    output = centerFinder.model.neck.poolformer_forward(ct_feat, center_pos_embedding)
    num_tasks = centerFinder.model.neck.tasks
    obj_num = centerFinder.model.neck.obj_num
    for idx, task in enumerate(num_tasks):
        out_dict_list[idx]["ct_feat"] = ct_feat[:, :, idx * obj_num : (idx+1) * obj_num]
    
with open(pickle_dir + "poolformer_output.pkl", 'rb') as handle:
    output_poolformer = pickle.load(handle)
with open(pickle_dir + "neck_output.pkl", 'rb') as handle:
    neck_output = pickle.load(handle)
    
print((output_poolformer[0][0][:50] == output[0][0][:50]).all())
print((out_dict_list[0]['hm']== neck_output[0]['hm']).all())

tensor(True, device='cuda:0')
tensor(True, device='cuda:0')


In [8]:
with torch.no_grad():
    preds = centerFinder.model.bbox_head(neck_output)
    
with open(pickle_dir + "bbox_head_output.pkl", 'rb') as handle:
    bbox_head_output = pickle.load(handle)
print((preds[0]['ct_feat'] == bbox_head_output[0]['ct_feat']).all())

tensor(True, device='cuda:0')


In [9]:
with torch.no_grad():
    output = model.bbox_head.predict(example, preds, model.test_cfg)

with open(pickle_dir + "predict_output.pkl", 'rb') as handle:
    predict_output = pickle.load(handle)

print((output[0]['box3d_lidar'] ==predict_output[0]['box3d_lidar']).all())
print((output[0]['scores'] ==predict_output[0]['scores']).all())
print((output[0]['label_preds'] ==predict_output[0]['label_preds']).all())

tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
